In [1]:
# Downloaded libraries
import torch
from torch import nn
from torch_geometric.loader import DataLoader


# Local files
from dataset_graphs import NNDataset
from models import Trainer_GCN

c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Constants
TRAINING_SPLIT = 0.8

In [3]:
# Hyperparameters
num_epoch = 50
batch_size = 16

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
nndataset = NNDataset(root="../")

size = len(nndataset)
train_num = int(size * TRAINING_SPLIT)
test_num = size - train_num

train_loader = DataLoader(
    dataset=nndataset[:train_num], batch_size=batch_size, shuffle=True)
test_loader = DataLoader(
    dataset=nndataset[train_num:], batch_size=test_num, shuffle=True)

print(
    f"Dataset loaded, {train_num} training samples and {test_num} testing samples")


Dataset loaded, 800 training samples and 200 testing samples


In [13]:
data = iter(train_loader).next()
print(data)

DataBatch(design=[16], edge_index=[2, 32240], x=[1057, 500], y_edge=[16120, 1], y_node=[1057, 1], input_mask=[1057, 1], num_nodes=1057, batch=[1057], ptr=[17])


In [7]:
model = Trainer_GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
loss_fn = nn.MSELoss()

In [8]:
# Model Training
model.train()
for epoch in range(num_epoch):
    print(f"Epoch {epoch + 1} / {num_epoch}:")
    for i, data in enumerate(train_loader):
        data.to(device)

        # forward propagation
        out_w, out_b = model(data)
        loss = loss_fn(out_b, data.y_node) + loss_fn(out_w, data.y_edge)

        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print status every n batches
        if i % 10 == 0:
            loss, current = loss.item(), i * batch_size
            print(
                f"Training Loss: {loss:>7f}  [{current:>5d}/{train_num:>5d}]")

Epoch 1 / 50:
Training Loss: 0.350593  [    0/  800]
Training Loss: 0.353456  [  160/  800]
Training Loss: 0.350117  [  320/  800]
Training Loss: 0.446917  [  480/  800]
Training Loss: 0.240783  [  640/  800]
Epoch 2 / 50:
Training Loss: 0.385168  [    0/  800]
Training Loss: 0.417437  [  160/  800]
Training Loss: 0.338012  [  320/  800]
Training Loss: 0.294482  [  480/  800]
Training Loss: 0.342864  [  640/  800]
Epoch 3 / 50:
Training Loss: 0.281939  [    0/  800]
Training Loss: 0.389644  [  160/  800]
Training Loss: 0.435217  [  320/  800]
Training Loss: 0.277769  [  480/  800]
Training Loss: 0.267295  [  640/  800]
Epoch 4 / 50:
Training Loss: 0.221983  [    0/  800]
Training Loss: 0.317485  [  160/  800]
Training Loss: 0.264799  [  320/  800]
Training Loss: 0.298477  [  480/  800]
Training Loss: 0.253460  [  640/  800]
Epoch 5 / 50:
Training Loss: 0.298702  [    0/  800]
Training Loss: 0.240750  [  160/  800]
Training Loss: 0.309719  [  320/  800]
Training Loss: 0.331750  [  480/ 

In [10]:
# Model Evaluation
model.eval()
with torch.no_grad():
    data = iter(test_loader).next().to(device)

    # forward propagation
    out_w, out_b = model(data)
    loss = loss_fn(out_b, data.y_node) + loss_fn(out_w, data.y_edge)

    loss = loss.item()
    print(f"Validation Loss: {loss:>7f}")


Validation Loss: 0.249114


In [15]:
data = data.to(device)
out_w, out_b = model(data)

In [18]:
torch.save(model.state_dict(), "../model/model")
print("Model saved")

Model saved
